In [1]:
import os
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import dspy


with open("xai_key.txt") as f:
    api_key = f.read().strip()

lm = dspy.LM('xai/grok-3-mini', api_key=api_key)
dspy.configure(lm=lm)


HW3_ROOT = os.getcwd()   
PRAG_DATA_DIR = os.path.join(HW3_ROOT, "PragmatiCQA", "data")
SOURCES_DIR   = os.path.join(HW3_ROOT, "PragmatiCQA-sources")

VAL_JSONL   = os.path.join(PRAG_DATA_DIR, "val.jsonl")
TRAIN_JSONL = os.path.join(PRAG_DATA_DIR, "train.jsonl")
TEST_JSONL  = os.path.join(PRAG_DATA_DIR, "test.jsonl")



TOP_K_RETRIEVE = 5 
SEED = 42

@dataclass
class FirstTurnExample:
    topic: str
    question: str
    gold_answer: str                  
    literal_spans: List[str]           
    pragmatic_spans: List[str]        
    conversation_id: str               

def read_jsonl(path: str) -> List[Dict]:
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    return rows


In [2]:

from typing import Any

def _extract_span_texts(objs: Optional[List[Dict[str, Any]]]) -> List[str]:
    """Safely extract the 'text' field from a_meta.literal_obj / pragmatic_obj entries."""
    if not objs:
        return []
    out = []
    for obj in objs:
        txt = obj.get("text")
        if isinstance(txt, str) and txt.strip():
            out.append(txt.strip())
    return out

def load_first_questions(val_jsonl_path: str) -> List[FirstTurnExample]:
    rows = read_jsonl(val_jsonl_path)
    examples: List[FirstTurnExample] = []

    for i, conv in enumerate(rows):
        topic = conv.get("topic", "").strip()
        qas = conv.get("qas") or []
        if not qas:
            continue  

        first = qas[0]
        question = (first.get("q") or "").strip()
        gold_answer = (first.get("a") or "").strip()

        a_meta = first.get("a_meta") or {}
        literal_objs = a_meta.get("literal_obj") or []
        pragmatic_objs = a_meta.get("pragmatic_obj") or []

        literal_spans = _extract_span_texts(literal_objs)
        pragmatic_spans = _extract_span_texts(pragmatic_objs)

        ex = FirstTurnExample(
            topic=topic or f"[UNKNOWN_TOPIC_{i}]",
            question=question,
            gold_answer=gold_answer,
            literal_spans=literal_spans,
            pragmatic_spans=pragmatic_spans,
            conversation_id=str(i),
        )
        examples.append(ex)

    return examples

val_first_q_examples = load_first_questions(VAL_JSONL)
print(f"[INFO] Loaded {len(val_first_q_examples)} first-question examples from val.jsonl")
if val_first_q_examples:
    e0 = val_first_q_examples[0]
    print("Sample:")
    print("topic:", e0.topic)
    print("question:", e0.question[:120], "...")
    print("gold_answer:", e0.gold_answer[:120], "...")
    print("#literal_spans:", len(e0.literal_spans), "| #pragmatic_spans:", len(e0.pragmatic_spans))


[INFO] Loaded 179 first-question examples from val.jsonl
Sample:
topic: A Nightmare on Elm Street (2010 film)
question: who is freddy krueger? ...
gold_answer: Freddy Kruger is the nightmare in nighmare on Elm street. Please note, and to be very clear, the system that loads up wi ...
#literal_spans: 1 | #pragmatic_spans: 1


In [3]:
# === Cell 3 (Unified): Topic resolver + FAISS retriever ===
from typing import List, Tuple, Optional
import os, re, glob, difflib
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# ----------------------------------------------------
EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL_NAME)

_topic_to_index: dict[str, faiss.Index] = {}
_topic_to_chunks: dict[str, List[str]] = {}

# ----------------------------------------------------
def normalize_topic_to_dirname(topic: str) -> str:
    t = topic.strip()
    t = re.sub(r"\s*\([^)]*\)", "", t)          
    t = re.sub(r"[^0-9A-Za-z _-]+", " ", t)    
    t = re.sub(r"\s+", " ", t).strip()
    t = t.replace(" ", "_")
    return t

def list_available_topics_dirs() -> list:
    if not os.path.isdir(SOURCES_DIR):
        print(f"[ERROR] Sources dir missing: {SOURCES_DIR}")
        return []
    return [d for d in os.listdir(SOURCES_DIR) if os.path.isdir(os.path.join(SOURCES_DIR, d))]

def resolve_topic_dir(topic: str) -> Optional[str]:
    candidates = list_available_topics_dirs()
    if not candidates:
        return None

    norm = normalize_topic_to_dirname(topic)
    direct = os.path.join(SOURCES_DIR, norm)
    if os.path.isdir(direct):
        return direct

    match = difflib.get_close_matches(norm, candidates, n=1, cutoff=0.6)
    if match:
        print(f"[INFO] Fuzzy-resolved topic '{topic}' -> '{match[0]}'")
        return os.path.join(SOURCES_DIR, match[0])

    raw = topic.replace(" ", "_")
    raw_match = difflib.get_close_matches(raw, candidates, n=1, cutoff=0.6)
    if raw_match:
        print(f"[INFO] Fuzzy-resolved (raw) '{topic}' -> '{raw_match[0]}'")
        return os.path.join(SOURCES_DIR, raw_match[0])

    print(f"[WARN] Could not resolve topic folder for: {topic} (norm='{norm}')")
    return None

# ----------------------------------------------------
def build_faiss_index_for_topic(topic: str) -> None:
    topic_dir = resolve_topic_dir(topic)
    if not topic_dir:
        print(f"[WARN] Topic dir not found for topic: {topic}")
        return

    html_files = glob.glob(os.path.join(topic_dir, "*.html"))
    chunks: List[str] = []
    for f in html_files:
        with open(f, "r", encoding="utf-8", errors="ignore") as fh:
            text = fh.read()
        for para in text.split("\n"):
            para = para.strip()
            if len(para) > 30:
                chunks.append(para)

    if not chunks:
        print(f"[WARN] No chunks for topic '{topic}' (dir='{topic_dir}')")
        return

    X = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    faiss.normalize_L2(X)
    index = faiss.IndexFlatIP(X.shape[1]) 
    index.add(X)

    _topic_to_index[topic] = index
    _topic_to_chunks[topic] = chunks
    print(f"[INFO] Built FAISS index | topic='{topic}' | dir='{os.path.basename(topic_dir)}' | #chunks={len(chunks)}")

def retrieve_context(topic: str, question: str, top_k: int = TOP_K_RETRIEVE) -> List[str]:
    resolved_dir = resolve_topic_dir(topic)
    if not resolved_dir:
        print(f"[WARN] Could not resolve dir for topic: {topic}")

    if topic not in _topic_to_index:
        build_faiss_index_for_topic(topic)
    if topic not in _topic_to_index:
        return []

    index = _topic_to_index[topic]
    chunks = _topic_to_chunks[topic]

    q_emb = embedder.encode([question], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)

    return [chunks[i] for i in I[0] if 0 <= i < len(chunks)]

sample_ex = val_first_q_examples[0]
print("Topic:", sample_ex.topic)
ctx = retrieve_context(sample_ex.topic, sample_ex.question, top_k=3)



Topic: A Nightmare on Elm Street (2010 film)
[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street
[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Built FAISS index | topic='A Nightmare on Elm Street (2010 film)' | dir='A Nightmare on Elm Street' | #chunks=33827


In [4]:
import dspy
from typing import List, Tuple, Optional
class SummarizeGoal(dspy.Signature):
    """Summarize the student's long-term goal or interests from prior Q/A history."""
    history = dspy.InputField(desc="List of prior (question, answer) pairs, oldest→newest.")
    summary = dspy.OutputField(desc="2-4 concise sentences summarizing goals/interests.")

class InferPragmaticNeed(dspy.Signature):
    """Infer the current pragmatic need behind the student's question."""
    question = dspy.InputField()
    history_summary = dspy.InputField()
    retrieved_glimpse = dspy.InputField(desc="Short excerpt(s) of the retrieved context.")
    need = dspy.OutputField(desc="Crisp statement of what extra info would be most helpful now.")

class GenerateCoopQuery(dspy.Signature):
    """Generate a cooperative follow-up retrieval query to fetch complementary context."""
    question = dspy.InputField()
    pragmatic_need = dspy.InputField()
    coop_query = dspy.OutputField(desc="A single focused query for the retriever (<= 160 chars).")

class ReasonCoT(dspy.Signature):
    """Deliberate on how to craft a cooperative answer grounded in the retrieved evidence."""
    question = dspy.InputField()
    history_summary = dspy.InputField()
    all_context = dspy.InputField(desc="Concise merged evidence snippets to ground the answer.")
    pragmatic_need = dspy.InputField()
    reasoning = dspy.OutputField(desc="Step-by-step plan: literal facts to include + helpful extras.")

class CooperativeAnswer(dspy.Signature):
    """Produce a cooperative answer grounded in retrieved evidence."""
    question = dspy.InputField()
    history_summary = dspy.InputField()
    all_context = dspy.InputField()
    reasoning = dspy.InputField()
    answer = dspy.OutputField(desc="Final cooperative answer. Cite evidence implicitly; avoid hallucinations; be concise and helpful.")


# ----------------------------------------------------
def _format_history(history: Optional[List[Tuple[str, str]]]) -> str:
    if not history:
        return "(no prior turns)"
    lines = []
    for i, (q, a) in enumerate(history, 1):
        lines.append(f"Turn {i} - Q: {q}\nTurn {i} - A: {a}")
    return "\n".join(lines)

def _shorten_chunks(chunks: List[str], max_chars: int = 1000) -> str:
    """Concatenate chunks with a soft character budget (for prompting)."""
    out, used = [], 0
    for c in chunks:
        c = c.strip()
        if not c:
            continue
        if used + len(c) + 2 > max_chars:
            break
        out.append(c)
        used += len(c) + 2
    return "\n---\n".join(out)


# ----------------------------------------------------
class MultiStepCoopQAModule(dspy.Module):
    def __init__(self, retriever_fn, top_k: int = 5, second_hop: bool = True):
        """
        retriever_fn: callable(topic: str, question: str, top_k: int) -> List[str]
        top_k: כמה קטעים לשלוף בכל שאילתה.
        second_hop: האם לבצע שליפה שניה עם ה-coop query.
        """
        super().__init__()
        self.retriever_fn = retriever_fn
        self.top_k = top_k
        self.second_hop = second_hop

        self.summarize = dspy.Predict(SummarizeGoal)
        self.infer_need = dspy.Predict(InferPragmaticNeed)
        self.gen_query = dspy.Predict(GenerateCoopQuery)
        self.reason = dspy.Predict(ReasonCoT)
        self.answer = dspy.Predict(CooperativeAnswer)

    def forward(
        self,
        *,
        topic: str,
        question: str,
        history: Optional[List[Tuple[str, str]]] = None,
        initial_context: Optional[List[str]] = None,
        top_k: Optional[int] = None,
    ):
        """
        API ראשי:
          - topic: שם הטופיק (לשליפה מתוך ה-HTMLים שלו)
          - question: השאלה הנוכחית
          - history: [(q,a), ...] (ב-4.4.1 לרוב None)
          - initial_context: אם כבר שלפת קטעים מראש (אפשר להשאיר None)
          - top_k: גובר על self.top_k אם סופק
        מחזיר: dict עם שלבי ביניים + answer סופי.
        """
        k = top_k or self.top_k

        ctx1 = initial_context
        if ctx1 is None:
            ctx1 = self.retriever_fn(topic, question, top_k=k)
        ctx1_short = _shorten_chunks(ctx1, max_chars=800) if ctx1 else "(no context retrieved)"

        history_txt = _format_history(history)

        summary = self.summarize(history=history_txt).summary
        need = self.infer_need(
            question=question,
            history_summary=summary,
            retrieved_glimpse=ctx1_short
        ).need

        coop_q = self.gen_query(
            question=question,
            pragmatic_need=need
        ).coop_query

        ctx_all = list(ctx1) if ctx1 else []
        if self.second_hop and coop_q and isinstance(coop_q, str) and len(coop_q.strip()) > 0:
            ctx2 = self.retriever_fn(topic, coop_q.strip(), top_k=max(2, k // 2))
            if ctx2:
                ctx_all.extend(ctx2)

        ctx_all_short = _shorten_chunks(ctx_all, max_chars=1600) if ctx_all else ctx1_short
        plan = self.reason(
            question=question,
            history_summary=summary,
            all_context=ctx_all_short,
            pragmatic_need=need
        ).reasoning

        final = self.answer(
            question=question,
            history_summary=summary,
            all_context=ctx_all_short,
            reasoning=plan
        ).answer

        return {
            "history_summary": summary,
            "pragmatic_need": need,
            "coop_query": coop_q,
            "context_1": ctx1[:k] if ctx1 else [],
            "context_all": ctx_all[: (k + max(2, k // 2)) ],
            "reasoning": plan,
            "answer": final,
        }


# ----------------------------------------------------
multi_step = MultiStepCoopQAModule(retriever_fn=retrieve_context, top_k=TOP_K_RETRIEVE, second_hop=True)

probe = multi_step(
    topic=val_first_q_examples[0].topic,
    question=val_first_q_examples[0].question,
    history=None,               
    initial_context=None,       
    top_k=3
)
print("Coop Query:", probe["coop_query"])
print("Answer (preview):", (probe["answer"] or "")[:300], "...")


[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street
[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street
Coop Query: Detailed biography of Freddy Krueger: origins in A Nightmare on Elm Street, creator, characteristics, and cultural significance.
Answer (preview): Freddy Krueger, whose full name is Frederick Charles "Freddy" Krueger, is a fictional character from the "A Nightmare on Elm Street" horror film franchise. He is often referred to as Fred Krueger and is portrayed as a vengeful dream-haunting antagonist with a burned appearance and a signature clawed ...


In [5]:
# === Cell 5: Run Multi-Step module on first questions and save outputs ===
import os, json, time
from tqdm import tqdm
from typing import Dict, Any, List, Optional

OUT_DIR = os.path.join(HW3_ROOT, "outputs")
os.makedirs(OUT_DIR, exist_ok=True)
PRED_JSONL = os.path.join(OUT_DIR, "val_firstq_multistep_predictions.jsonl")

def run_first_questions_multistep(
    examples: List[FirstTurnExample],
    module: MultiStepCoopQAModule,
    top_k: int = TOP_K_RETRIEVE,
    limit: Optional[int] = None,
    save_path: Optional[str] = None,
) -> List[Dict[str, Any]]:

    results: List[Dict[str, Any]] = []
    N = len(examples) if limit is None else min(limit, len(examples))
    t0 = time.time()

    for ex in tqdm(examples[:N], total=N, desc="MultiStep on first questions"):
        try:
            initial_ctx = retrieve_context(ex.topic, ex.question, top_k=top_k)

            out = module(
                topic=ex.topic,
                question=ex.question,
                history=None,             
                initial_context=initial_ctx,
                top_k=top_k
            )

            rec: Dict[str, Any] = {
                "conversation_id": ex.conversation_id,
                "topic": ex.topic,
                "question": ex.question,
                "pred_answer": out.get("answer", ""),
                "gold_answer": ex.gold_answer,
                "literal_spans": ex.literal_spans,
                "pragmatic_spans": ex.pragmatic_spans,
                "history_summary": out.get("history_summary", ""),
                "pragmatic_need": out.get("pragmatic_need", ""),
                "coop_query": out.get("coop_query", ""),
                "context_1": out.get("context_1", []),
                "context_all": out.get("context_all", []),
                "reasoning": out.get("reasoning", ""),
            }
        except Exception as e:
            rec = {
                "conversation_id": ex.conversation_id,
                "topic": ex.topic,
                "question": ex.question,
                "pred_answer": "",
                "gold_answer": ex.gold_answer,
                "literal_spans": ex.literal_spans,
                "pragmatic_spans": ex.pragmatic_spans,
                "error": str(e),
            }
        results.append(rec)

        if save_path:
            with open(save_path, "a", encoding="utf-8") as f:
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    dt = time.time() - t0
    print(f"[INFO] Done {N} examples in {dt:.1f}s ({dt/max(N,1):.2f}s/ex)")
    print(f"[INFO] Saved to: {save_path}" if save_path else "[INFO] Not saved (save_path=None)")
    return results

if os.path.exists(PRED_JSONL):
    os.remove(PRED_JSONL)

val_firstq_preds = run_first_questions_multistep(
    val_first_q_examples,
    multi_step,
    top_k=TOP_K_RETRIEVE,
    limit=None,                   
    save_path=PRED_JSONL
)

print(f"Total predictions: {len(val_firstq_preds)}")
for rec in val_firstq_preds[:2]:
    print("-"*80)
    print("Q:", rec["question"])
    print("Pred:", (rec["pred_answer"] or "")[:300], "...")
    print("Gold:", (rec["gold_answer"] or "")[:300], "...")
    print("Coop Query:", rec.get("coop_query"))


MultiStep on first questions:   0%|          | 0/179 [00:00<?, ?it/s]

[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street
[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street


MultiStep on first questions:   1%|          | 1/179 [00:26<1:19:09, 26.68s/it]

[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street
[INFO] Fuzzy-resolved topic 'A Nightmare on Elm Street (2010 film)' -> 'A Nightmare on Elm Street'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\A Nightmare on Elm Street


MultiStep on first questions:   1%|          | 2/179 [00:49<1:12:44, 24.66s/it]

...

[INFO] Fuzzy-resolved topic 'Game of Thrones' -> 'Game of Thrones'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\Game of Thrones
[INFO] Fuzzy-resolved topic 'Game of Thrones' -> 'Game of Thrones'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\Game of Thrones


MultiStep on first questions:  99%|█████████▉| 178/179 [3:45:37<00:28, 28.84s/it]

[INFO] Fuzzy-resolved topic 'Game of Thrones' -> 'Game of Thrones'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\Game of Thrones
[INFO] Fuzzy-resolved topic 'Game of Thrones' -> 'Game of Thrones'
[INFO] Using topic dir: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\PragmatiCQA-sources\Game of Thrones


MultiStep on first questions: 100%|██████████| 179/179 [3:46:02<00:00, 75.77s/it]

[INFO] Done 179 examples in 13562.5s (75.77s/ex)
[INFO] Saved to: c:\Users\gilic\hw3\nlp-with-llms-2025-hw3\outputs\val_firstq_multistep_predictions.jsonl
Total predictions: 179
--------------------------------------------------------------------------------
Q: who is freddy krueger?
Pred: Freddy Krueger, whose full name is Frederick Charles "Freddy" Krueger, is the central antagonist in the "A Nightmare on Elm Street" film series. He was a notorious child murderer with pre-death victims, who was killed by vigilante parents and later returned as a supernatural entity that haunts and k ...
Gold: Freddy Kruger is the nightmare in nighmare on Elm street. Please note, and to be very clear, the system that loads up wiki is not allowing access to Adam Prag, to the page... so I'll have to go from memory.  Normally you can paste things and back up what you are saying, but today that's not happenin ...
Coop Query: Comprehensive biography of Freddy Krueger: origins, characteristics, role in A Ni

In [13]:
from dspy.evaluate import SemanticF1
import pandas as pd

metric = SemanticF1()

def prf1(q, gold, pred):
    s = metric.module(
        question=q or "",
        ground_truth=str(gold or ""),
        system_response=str(pred or "")
    )
    p, r = float(s.precision), float(s.recall)
    f1 = 2 * p * r / (p + r + 1e-9)
    return p, r, f1

p_vals, r_vals, f1_vals = zip(*[
    prf1(r["question"], r["gold_answer"], r["pred_answer"])
    for r in val_firstq_preds
])

scores_df = pd.DataFrame({
    "topic":    [r.get("topic", "")    for r in val_firstq_preds],
    "question": [r.get("question", "") for r in val_firstq_preds],
    "gold":     [r.get("gold_answer")  for r in val_firstq_preds],
    "pred":     [r.get("pred_answer")  for r in val_firstq_preds],
    "precision": p_vals,
    "recall":    r_vals,
    "f1":        f1_vals,
})

display(scores_df[["precision", "recall", "f1"]].astype(float).describe())
scores_df.to_csv("part4_4_1_metrics.csv", index=False)


,precision,recall,f1
count,179.000000,179.000000,179.000000
mean,0.234178,0.361455,0.260406
std,0.211046,0.320411,0.217244
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.200000,0.330000,0.250000
75%,0.333000,0.500000,0.400000
max,0.800000,1.000000,0.857143


In [ ]:
import json
from pathlib import Path
from typing import List, Dict, Any

def load_all_turns(val_jsonl_path: Path) -> List[Dict[str, Any]]:
    all_data = []
    with open(val_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            convo = json.loads(line)
            topic = convo.get("topic", "")
            qas = convo.get("qas", [])

            all_data.append({
                "topic": topic,
                "qas": qas
            })
    return all_data


In [ ]:
val_jsonl_path = Path("/PragmatiCQA/data/val.jsonl") 
val_conversations = load_all_turns(val_jsonl_path)
